In [1]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
working_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)

In [2]:
# Personnal import 
from utils.utilities_DL import match_period_coverage_with_netmob,get_small_ds
from constants.config import get_args,update_modif
from constants.paths import folder_path,file_name,SAVE_DIRECTORY
from K_fold_validation.K_fold_validation import KFoldSplitter

# Hp Tuning
from HP_tuning.hyperparameter_tuning_ray import HP_tuning


'pynvml' is not available on this environment.


In [3]:
# Load config
model_name = 'STGCN' #'CNN'
args = get_args(model_name)

# Modification : 
args.K_fold = 5
args.ray = True
args.W = 0  # IMPORTANT AVEC NETMOB
args.epochs = 10
args.loss_function_type = 'MSE' # 'quantile'

args = update_modif(args)

# Coverage Period : 
small_ds = False
coverage = match_period_coverage_with_netmob(file_name)
(coverage,args) = get_small_ds(small_ds,coverage,args)

# Choose DataSet and VisionModel if needed: 
dataset_names = ['subway_in'] # ['calendar','netmob'] #['subway_in','netmob','calendar']
vision_model_name = 'ImageAvgPooling'  # 'ImageAvgPooling'  #'FeatureExtractor_ResNetInspired' #'MinimalFeatureExtractor',

# Train and Evaluate Model: 
mod_plot = 1 # bokeh plotting every epoch 

# Load K-fold subway-ds 
folds = [0] # Here we use the first fold for HP-tuning. 

# In case we need to compute the Sliding K-fold validation:
# folds = np.arange(1,args.K_fold)

K_fold_splitter = KFoldSplitter(dataset_names,args,coverage,folder_path,file_name,vision_model_name,folds)
K_subway_ds,args_embedding,args_vision,dic_class2rpz = K_fold_splitter.split_k_fold()

Model: STGCN, K_fold = 5
!!! Loss function: MSE 
Time-step per hour: 4.0
coverage period: 2019-03-16 00:00:00 - 2019-05-31 23:45:00

Init Subway-In Dataset:  torch.Size([7392, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295680 


U size:  torch.Size([6719, 40, 7]) Utarget size:  torch.Size([6719, 40, 1])
U_train size:  torch.Size([3912, 40, 7]) Utarget_train size:  torch.Size([3912, 40, 1])
U_valid size:  torch.Size([1304, 40, 7]) Utarget_valid size:  torch.Size([1304, 40, 1])
U_test size:  torch.Size([1310, 40, 7]) Utarget_test size:  torch.Size([1310, 40, 1])
U_train min:  tensor(0.) U_train max:  tensor(10798.)
U_valid min:  tensor(0.) U_valid max:  tensor(1405.)
U_test min:  tensor(0.) U_test max:  tensor(1760.)

 ===== ERROR ==== 
Try with torch >= 2.0.0 (works with 2.0.1) 
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing

 ===== ERROR ==== 
Try with torch >= 2.0.0 (works with 2.0.1) 
Va

## HP Tuning on Fold0:

In [ ]:
num_samples = 8
subway_ds = K_subway_ds[0]
analysis = HP_tuning(subway_ds,args,args_embedding,args_vision,num_samples,dic_class2rpz,working_dir)

2024-08-19 14:58:51,631	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2024-08-19 14:58:52,249	INFO packaging.py:518 -- Creating a file package for local directory '/home/rrochas/prediction_validation'.
2024-08-19 14:58:52,344	WARNING packaging.py:393 -- File /home/rrochas/prediction_validation/.git/objects/6a/2f986b4cfd0d5c1b5370539c60cbc60376ee7c is very large (20.29MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['/home/rrochas/prediction_validation/.git/objects/6a/2f986b4cfd0d5c1b5370539c60cbc60376ee7c']})`
2024-08-19 14:58:52,571	WARNING packaging.py:393 -- File /home/rrochas/prediction_validation/.git/objects/62/340d41856c322da363f152c93b2cc7ca7e1b52 is very large (17.61MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['/home/rrochas/prediction_validation/.git/objects/62/340d41856c322da363f152c9

(raylet) [2024-08-19 14:59:11,648 E 136455 136468] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-19_14-58-49_384821_136209 is over 95% full, available space: 1259237376; capacity: 426114023424. Object creation will fail if spilling is required.
2024-08-19 14:59:16,706	WARNING worker.py:2037 -- Warning: The actor ImplicitFunc is very large (26 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(raylet) [2024-08-19 14:59:21,670 E 136455 136468] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-19_14-58-49_384821_136209 is over 95% full, available space: 1313226752; capacity: 426114023424. Object creation will fail if spilling is required.


(pid=138139) 'pynvml' is not available on this environment.
(<lambda> pid=138139) number of total parameters: 219649
(<lambda> pid=138139) number of trainable parameters: 219649
(<lambda> pid=138139) 
(<lambda> pid=138139) start training
(pid=138135) 'pynvml' is not available on this environment. [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)


(raylet) [2024-08-19 14:59:31,679 E 136455 136468] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-19_14-58-49_384821_136209 is over 95% full, available space: 1312088064; capacity: 426114023424. Object creation will fail if spilling is required.


Trial name,_metric
lambda_d2f63_00000,{'Loss_model': 0.031565853140026426}
lambda_d2f63_00001,{'Loss_model': 0.03560343143976386}
lambda_d2f63_00002,{'Loss_model': 0.038925094961209696}
lambda_d2f63_00003,{'Loss_model': 0.03508422622558963}


(raylet) [2024-08-19 14:59:41,714 E 136455 136468] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-19_14-58-49_384821_136209 is over 95% full, available space: 1311768576; capacity: 426114023424. Object creation will fail if spilling is required.
(raylet) [2024-08-19 14:59:51,737 E 136455 136468] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-19_14-58-49_384821_136209 is over 95% full, available space: 1312145408; capacity: 426114023424. Object creation will fail if spilling is required.


(<lambda> pid=138135) number of total parameters: 294593 [repeated 3x across cluster]
(<lambda> pid=138135) number of trainable parameters: 294593 [repeated 3x across cluster]
(<lambda> pid=138135)  [repeated 3x across cluster]
(<lambda> pid=138135) start training [repeated 3x across cluster]
(pid=138353) 'pynvml' is not available on this environment.


(<lambda> pid=138353) /root/anaconda3/envs/tuning_rochas/lib/python3.11/site-packages/scipy/sparse/linalg/_eigen/_svds.py:482: UserWarning: Exited at iteration 20 with accuracies 
(<lambda> pid=138353) [0.01494232]
(<lambda> pid=138353) not reaching the requested tolerance 5.960464477539062e-07.
(<lambda> pid=138353) Use iteration 20 instead with accuracy 
(<lambda> pid=138353) 0.013802357586460439.
(<lambda> pid=138353) 
(<lambda> pid=138353)   _, eigvec = lobpcg(XH_X, X, tol=tol ** 2, maxiter=maxiter,
(<lambda> pid=138353) /root/anaconda3/envs/tuning_rochas/lib/python3.11/site-packages/scipy/sparse/linalg/_eigen/_svds.py:482: UserWarning: Exited postprocessing with accuracies 
(<lambda> pid=138353) [0.01380236]
(<lambda> pid=138353) not reaching the requested tolerance 5.960464477539062e-07.
(<lambda> pid=138353)   _, eigvec = lobpcg(XH_X, X, tol=tol ** 2, maxiter=maxiter,


## Classical Training: 

In [11]:
from trainer import Trainer
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from plotting.plotting_bokeh import plot_bokeh

subway_ds = K_subway_ds[0]
args.epochs = 100
# Load Model:
model = load_model(args,args_embedding,dic_class2rpz,args_vision)

# Load Optimizer, Scheduler, Loss function: 
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)

trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = scheduler,args_embedding  =args_embedding,dic_class2rpz = dic_class2rpz,show_figure = True)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
# Train Model 
trainer.train_and_valid(mod = 1000,mod_plot = None)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

pi,pi_cqr = plot_bokeh(trainer,subway_ds.normalizer,subway_ds.tensor_limits_keeper.df_verif_test,args)

number of total parameters: 207233
number of trainable parameters: 207233

start training


2024-08-17 19:38:02,630	WARNING session.py:91 -- Session not detected. You should not be calling `report` outside `tuner.fit()` or while using the class API. 
2024-08-17 19:38:02,631	WARNING session.py:97 --   File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/root/anaconda3/envs/tuning_rochas/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/root/anaconda3/envs/tuning_rochas/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/root/anaconda3/envs/tuning_rochas/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/root/anaconda3/envs/tuning_rochas/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "/root/anaconda3/envs/tuning_rochas/lib/python3.11/asyncio/base_events.py", line 608

epoch: 0 
 min\epoch : 0.01
Estimated time for training: 0.5min 


/root/anaconda3/envs/tuning_rochas/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


>>> Training complete in: 0:00:30.485056
>>> Training performance time: min 0.01663804054260254 avg 0.03587746620178223 seconds (+/- 0.0073763718830102765)
>>> Loading performance time: min 0.0001499652862548828 avg 0.02773769826836925 seconds (+/- 0.030250714234803)
>>> Forward performance time: 0.01972544026040362 seconds (+/- 0.005561620757702408)
>>> Backward performance time: 0.013436449792437659 seconds (+/- 0.002075444830227948)
>>> Plotting performance time: 2.7984079688486426e-06 seconds (+/- 5.610231861720418e-06)
>>> PI-tracking performance time: 0.024527378756590563 seconds (+/- 0.004512244056108801)
>>> Scheduler-update performance time: 4.177623324924045e-05 seconds (+/- 0.0001468469470304164)
>>> Validation time: 0:00:00.084269
Proportion of time consumed for Loading: 44.4%
Proportion of time consumed for Forward: 28.7%
Proportion of time consumed for Backward: 19.6%
Proportion of time consumed for Plotting: 0.0%
Proportion of time consumed for CheckPoint Saving: 0.0%
Pr

Loading BokehJS ...

In [4]:
args.Kt

3

In [ ]:
from utils.utilities_DL import get_MultiModel_loss_args_emb_opts,load_init_trainer


## Hyper Parameter Tuning sur le Fold 0
def load_trainer(config,folder_path,file_name,args):

    for key, value in config.items():
        if hasattr(args, key):
            setattr(args, key, value)

    Datasets,DataLoader_list,dic_class2rpz,nb_words_embedding,time_slots_labels,dic_rpz2class = load_init_trainer(folder_path,file_name,args)
    (loss_function,Model_list,Optimizer_list,Scheduler_list,args_embedding) = get_MultiModel_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz,n_vertex = len(Datasets[0].columns))
    dataset,dataloader,model,optimizer,scheduler = Datasets[0],DataLoader_list[0],Model_list[0],Optimizer_list[0],Scheduler_list[0]


    trainer = Trainer(dataset,model,
                    args,optimizer,loss_function,scheduler = scheduler,
                    args_embedding=args_embedding,dic_class2rpz=dic_class2rpz)
    return(trainer)

def Train_with_tune(config,folder_path,file_name,args):
    trainer = load_trainer(config,folder_path,file_name,args)
    result_df = trainer.train_and_valid()

    
def run_tuning_and_save_results(args,num_samples):
    config = get_search_space_ray(args)
    ray_scheduler,ray_search_alg,resources_per_trial,num_gpus,max_concurrent_trials,num_cpus = get_ray_config(args)
    
    def trainer(config):
        return(Train_with_tune(config,folder_path,file_name,args))
        
    if ray.is_initialized:
        ray.shutdown()
        ray.init(num_gpus=num_gpus,num_cpus=num_cpus)

        
    analysis = tune.run(
            trainer,
            config=config,
            num_samples=num_samples,  # Increase num_samples for more random combinations
            resources_per_trial = resources_per_trial,
            max_concurrent_trials = max_concurrent_trials,
            scheduler = ray_scheduler,
            search_alg = ray_search_alg,
        )

    name_save = f"HyperparameterTuning/Htuning_ray_analysis_{args.model_name}_loss{args.loss_function_type}_TE_{args.time_embedding}"
    analysis.results_df.to_csv(f'{name_save}.csv')
    

In [ ]:
# ==== GET PARAMETERS ====
# Load config
model_name = 'STGCN'  #'CNN'
args = get_args(model_name)

# Classic Modification : 
args.epochs = 1
args.loss_function_type = 'MSE' # 'quantile'
# ...

# Modification pour HyperParameterTuning:
args.K_fold = 5
args.ray = True
# ...

# Update Modif 
args = update_modif(args,name_gpu='cuda:0')
# ...

run_tuning_and_save_results(args,num_samples=1000)